# 1. Perkenalan

'''


Nama  : Adhi Rizqi Alfaqih

Program ini dibuat untuk melakukan pengambilan, pengolahan, dan analisis data.

'''

# 2. Bussiness Understanding

## 2.1. Latar Belakang

Dalam era digital saat ini, platform e-commerce semakin berkembang pesat. Dengan meningkatnya jumlah transaksi online, penting bagi perusahaan untuk memiliki sistem yang efisien dalam mengelola dan menganalisis data penjualan. Data warehouse menjadi solusi yang tepat untuk menyimpan dan menganalisis data ini, sehingga perusahaan dapat mengambil keputusan yang lebih baik berdasarkan informasi yang akurat.

## 2.2. Permasalahan Bisnis

Perusahaan e-commerce menghadapi beberapa tantangan dalam mengelola data penjualan, antara lain:

- Pengelolaan Data yang Terpisah: Data penjualan, pelanggan, dan produk sering kali disimpan dalam sistem yang terpisah, membuat analisis menjadi sulit.
- Kurangnya Wawasan: Tanpa analisis yang tepat, perusahaan tidak dapat memahami pola pembelian pelanggan, produk yang paling laku, dan tren penjualan.
- Kesulitan dalam Pelaporan: Proses pelaporan yang lambat dan tidak efisien dapat menghambat pengambilan keputusan yang cepat.


## 2.3. Proses Bisnis

Proses bisniss pada perusahaan e-commerce dalam mengelola data penjualan, antara lain:

- Pengguna / Pelanggan dapat mendaftarkan akunnya kepada 'The Look' lalu mengisi identitas seperti, "Nama Depan", "Nama Belakang", "Jenis Kelamin", "Email", serta "Alamat". Data-data tersebut akan dimasukkan ke Data "user"
- Penjual dapat mendaftarkan produknya kepada "The Look" lalu mengisi identitas produk seperti, "Nama Produk", "Merk", "Kategori", dan "Harga Jual". Data-data tersebut akan dimasukkan ke data "product"
- Lalu apabila terjadi transaksi antara Pelanggan dengan Penjual, data-data seperti "user", "product", "tanggal pemesanan", dan "status pengiriman" akan masuk ke data "orders"
- Untuk mempermudah presentasi penjualan, kita akan membuat filter data berupa tanggal, yang berisi "Tanggal", "Bulan", "Quarter", "Half Year" dan "Year". Kita masukkan data-data tersebut menjadi data "date".


# 3. Step of Dimensional Modelling

## 3.1. Identify Business Process

Fokus utama dari data warehouse ini adalah proses penjualan, yang mencakup pengelolaan pesanan, item dalam pesanan, dan informasi produk serta pengguna.



## 3.2. Identify Grain (Level of Detail)

Grain dari data warehouse ini adalah transaksi penjualan, yang mencakup setiap item yang dipesan dalam setiap pesanan. Setiap baris dalam fact table akan mewakili satu item dalam satu pesanan.

## 3.3. Identify Dimension

Berdasarkan tabel yang ada, berikut adalah dimension tables yang dapat diidentifikasi:

a. Dimension Table: users_dimension

- Menyimpan informasi tentang pengguna.

b. Dimension Table: products_dimension

- Menyimpan informasi tentang produk.

c. Dimension Table: date_dimension

- Informasi filterisasi tanggal.


## 3.4. Identify Facts

Fact table akan menyimpan data transaksi penjualan. jadi akan diidentifikasikan sebagai :
- Fact Table : sales_fact

# 4. Data Loading

In [1]:
!pip install spark #Instalasi pyspark

In [2]:
import pyspark #Setelah instalasi, import pyspark ke dalam dataset

In [3]:
from pyspark.sql import SparkSession #Untuk bisa men synchrone kan pyspark dengan sql

In [4]:
from pyspark.sql import functions as F #untuk menambahkan function

In [5]:
#Fungsi Query yang diperlukan untuk proses ini
from pyspark.sql.functions import count, col, sum, monotonically_increasing_id, to_date, month, quarter, when,year

In [6]:
#membuat app name pada spark
spark = SparkSession.builder \
    .appName("WriteToPostgres") \
    .appName("DropDuplicatesExample") \
    .appName("FillMissingValues") \
    .appName("UniqueValuesExample") \
    .appName("RenameColumnsExample") \
    .appName("JoinExample") \
    .appName("RenameandMoveColumn") \
    .appName("JoinUserandProductbyUserID") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

In [7]:
# Akses SparkContext
sc = spark.sparkContext
# List the loaded JAR files
print("JAR Files:")
print(sc._jsc.sc().listJars())

JAR Files:
List(spark://effd6fc3b895:32893/jars/org.postgresql_postgresql-42.6.0.jar, spark://effd6fc3b895:32893/jars/org.checkerframework_checker-qual-3.31.0.jar)


## 4.1. users_dimension

### 4.1.1 Data Loading

In [8]:
#import data .csv ke dalam dataframe "user_dimension"
users_dimension = spark.read.csv('thelook_ecommerce.users.csv', header=True, inferSchema=True)

In [9]:
#menampilkan keseluruahn data "users_dimension"
users_dimension.show()

+-----+----------+---------+--------------------+---+------+-----+--------------------+-----------+----+-------+------------+------------+--------------+-------------------+--------------------+
|   id|first_name|last_name|               email|age|gender|state|      street_address|postal_code|city|country|    latitude|   longitude|traffic_source|         created_at|           user_geom|
+-----+----------+---------+--------------------+---+------+-----+--------------------+-----------+----+-------+------------+------------+--------------+-------------------+--------------------+
|80930|     Donna|Gutierrez|donnagutierrez@ex...| 34|     F| Acre|2319 Osborn Junct...|  69980-000|null| Brasil|-8.065346116|-72.87094866|        Search|2022-03-14 15:01:00|POINT(-72.8709486...|
|80559|  Jermaine|  Pittman|jermainepittman@e...| 45|     M| Acre|3750 Kim Mill Apt...|  69980-000|null| Brasil|-8.065346116|-72.87094866|        Search|2020-12-29 13:19:00|POINT(-72.8709486...|
|98339|     Henry|   Rive

In [10]:
#menampilkan informasi jenis data di "users_dimension"
users_dimension.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- state: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- user_geom: string (nullable = true)



In [11]:
#mengecek jumlah data yang "missing values" pada tiap kolom
users_dimension_mv = users_dimension.select([sum(col(c).isNull().cast("int")).alias(c) for c in users_dimension.columns])

In [12]:
#menampilkan hasil pengecekkan jumlah "missing values"
users_dimension_mv.show()

+---+----------+---------+-----+---+------+-----+--------------+-----------+----+-------+--------+---------+--------------+----------+---------+
| id|first_name|last_name|email|age|gender|state|street_address|postal_code|city|country|latitude|longitude|traffic_source|created_at|user_geom|
+---+----------+---------+-----+---+------+-----+--------------+-----------+----+-------+--------+---------+--------------+----------+---------+
|  0|         0|        0|    0|  0|     0|    0|             0|          0|   0|      0|       0|        0|             0|         0|        0|
+---+----------+---------+-----+---+------+-----+--------------+-----------+----+-------+--------+---------+--------------+----------+---------+



Dari data terlihat bahwa tidak ada "missing values" di dalam data "users_dimension"

### 4.1.2 Data Cleaning

In [13]:
#Kita drop untuk beberapa kolom yang tidak berhubungan dengan "penjualan"
users_dimension = users_dimension.drop("latitude","longitude","traffic_source","user_geom","age","state","street_address", "postal_code","city","country")

In [14]:
#kita drop sekiranya data yang memiliki data yang duplikat
users_dimension = users_dimension.dropDuplicates()

In [15]:
#merubah nama kolom "id" menjadi "user_id"
users_dimension = users_dimension.withColumnRenamed("id", "user_id")

In [16]:
#menampilkan hasil pembersihan dari "users_dimension"
users_dimension.show()

+-------+----------+----------+--------------------+------+-------------------+
|user_id|first_name| last_name|               email|gender|         created_at|
+-------+----------+----------+--------------------+------+-------------------+
|  75428|  Brittany|    Landry|brittanylandry@ex...|     F|2019-09-14 03:13:00|
|  50056|     Scott|  Figueroa|scottfigueroa@exa...|     M|2020-02-01 04:48:00|
|  80517|    Joseph|     Mills|josephmills@examp...|     M|2021-12-19 12:05:00|
|  73778|     Molly|   Fuentes|mollyfuentes@exam...|     F|2024-05-16 00:52:00|
|  87138|   Crystal|  Jacobson|crystaljacobson@e...|     F|2021-05-14 15:39:00|
|  48838|    Samuel|     Banks|samuelbanks@examp...|     M|2023-07-17 13:54:00|
|  27802|     James|    Cooper|jamescooper@examp...|     M|2022-12-01 02:05:00|
|  51643|     Laura|    Fisher|laurafisher@examp...|     F|2022-08-27 05:17:00|
|  80932|    Sandra|    Clarke|sandraclarke@exam...|     F|2023-08-17 07:28:00|
|  13038|      Lori|    Murray|lorimurra

## 4.2. products_dimension

### 4.2.1. Data Loading

In [17]:
#import data .csv ke dalam dataframe "product_dimension"
products_dimension = spark.read.csv('thelook_ecommerce.products.csv', header=True, inferSchema=True)

In [18]:
#menampilkan "product_dimension"
products_dimension.show()

+-----+------------------+-----------+--------------------+-----+------------------+----------+--------------------+----------------------+
|   id|              cost|   category|                name|brand|      retail_price|department|                 sku|distribution_center_id|
+-----+------------------+-----------+--------------------+-----+------------------+----------+--------------------+----------------------+
|13842| 2.518749990849756|Accessories|Low Profile Dyed ...|   MG|              6.25|     Women|EBD58B8A3F1D72F42...|                     1|
|13928|2.3383499148894105|Accessories|Low Profile Dyed ...|   MG| 5.949999809265137|     Women|2EAC42424D12436BD...|                     1|
|14115| 4.879559879379869|Accessories|Enzyme Regular So...|   MG|10.989999771118164|     Women|EE364229B2791D1EF...|                     1|
|14157| 4.648769887297898|Accessories|Enzyme Regular So...|   MG|10.989999771118164|     Women|00BD13095D06C20B1...|                     1|
|14273| 6.5079298864

In [19]:
#Untuk menampilkan informasi data di "product_dimension"
products_dimension.printSchema()

root
 |-- id: integer (nullable = true)
 |-- cost: double (nullable = true)
 |-- category: string (nullable = true)
 |-- name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- retail_price: double (nullable = true)
 |-- department: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- distribution_center_id: integer (nullable = true)



Dari data tersebut terdapat keganjalan, data "cost" ber tipe "double". Tipe tersebut terdapat kemungkinan yaitu Numeric dan STR. Kita akan buat menjadi data numeric

In [20]:
#untuk menghitung jumlah "missing values"
products_dimension_mv = products_dimension.select([sum(col(c).isNull().cast("int")).alias(c) for c in products_dimension.columns])

In [21]:
#menampilkan jumlah missing values
products_dimension_mv.show()

+---+----+--------+----+-----+------------+----------+---+----------------------+
| id|cost|category|name|brand|retail_price|department|sku|distribution_center_id|
+---+----+--------+----+-----+------------+----------+---+----------------------+
|  0|   0|       0|   2|   24|           0|         0|  0|                     0|
+---+----+--------+----+-----+------------+----------+---+----------------------+



Dari data "products_dimension_mv", terdapat 2 missing values di kolom "name" dan 24 missing values di kolom "brand"

### 4.2.2 Data Cleaning

In [22]:
#Data yang tidak berhubungan dengan dengan penjualan akan di drop
products_dimension = products_dimension.drop("distribution_center_id","sku","department")

In [23]:
#Kita akan ubah tipe data yang 'Double' pada kolom "cost' menjadi tipe 'float'
products_dimension = products_dimension.withColumn("cost", col("cost").cast("float"))

In [24]:
#Kita akan men drop data apabila terdapat data yang duplikat
products_dimension = products_dimension.dropDuplicates()

In [25]:
#Untuk mengisi missing values pada kolom "name" dan "brand"
products_dimension = products_dimension.fillna({
    'name': 'Unknown' , # Ganti dengan nilai 'unknown'
    'brand': 'Unknown'     # Ganti dengan nilai 'unknown'
})

In [26]:
#Mengubah nama kolom "id" menjadi "product_id"
products_dimension = products_dimension.withColumnRenamed("id", "product_id")

In [27]:
#mengecek kembali jumlah missing values
products_dimension_mv = products_dimension.select([sum(col(c).isNull().cast("int")).alias(c) for c in products_dimension.columns])

In [28]:
#menampilkan jumlah misisng values
products_dimension_mv.show()

+----------+----+--------+----+-----+------------+
|product_id|cost|category|name|brand|retail_price|
+----------+----+--------+----+-----+------------+
|         0|   0|       0|   0|    0|           0|
+----------+----+--------+----+-----+------------+



Dari data tersebut tidak ada missing values, memiliki arti yaitu proses pegisian missing values berhasil dilakukan

In [29]:
#Menampilkan hasil "products_dimension" setelah dicleaning
products_dimension.show()

+----------+---------+--------------------+--------------------+--------------+------------------+
|product_id|     cost|            category|                name|         brand|      retail_price|
+----------+---------+--------------------+--------------------+--------------+------------------+
|     18363| 65.09558|              Active|2XU Men's Compres...|           2XU|159.94000244140625|
|     12717|   46.728|                Swim|Echo Design Women...|          ECHO|             118.0|
|     28627|15.465581|         Accessories|Echo Design Men's...|          ECHO|  34.9900016784668|
|     22629|   30.745|               Pants| Matix Men's Mj Cord|         Matix|              65.0|
|      1821|   11.475|Fashion Hoodies &...|Hurley Juniors Yu...|        Hurley|              25.0|
|     22942|   26.433|              Shorts|Hurley Men's Puer...|        Hurley|              49.5|
|     28396|    9.675|         Accessories|Hurley Men's One ...|        Hurley|              25.0|
|     2058

## 4.3. date_dimension

### 4.3.1 Data Loading

Kita akan membuat data "date_dimension". Tentunya tidak ada pada file.csv, tentunya pada data BigQuery tidak tersedia, maka kita akan buat data itu sendiri, berdasarkan "created_at" dari data "order_items"

In [30]:
#Import file order_items.csv ke dalam dataframe
date_dimension = spark.read.csv('thelook_ecommerce.order_items.csv', header=True, inferSchema=True)

In [31]:
#Menampilkan isi dari order_items.csv
date_dimension.show()

+------+--------+-------+----------+-----------------+----------+-------------------+-------------------+-------------------+-----------+------------------+
|    id|order_id|user_id|product_id|inventory_item_id|    status|         created_at|         shipped_at|       delivered_at|returned_at|        sale_price|
+------+--------+-------+----------+-----------------+----------+-------------------+-------------------+-------------------+-----------+------------------+
|127348|   87852|  70237|     14235|           343297| Cancelled|2024-08-02 15:46:40|               NULL|               NULL|       NULL|0.0199999995529651|
| 68311|   47109|  37633|     14235|           184060|  Complete|2022-11-23 14:12:14|2022-11-25 11:43:00|2022-11-28 12:11:00|       NULL|0.0199999995529651|
| 61918|   42739|  34142|     14235|           166877|   Shipped|2022-06-03 02:10:00|2022-06-03 15:19:00|               NULL|       NULL|0.0199999995529651|
|161542|  111381|  89163|     14235|           435667|   S

In [32]:
#menampilkan jenis data pada "order_items.csv"
date_dimension.printSchema()

root
 |-- id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- inventory_item_id: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- shipped_at: timestamp (nullable = true)
 |-- delivered_at: timestamp (nullable = true)
 |-- returned_at: timestamp (nullable = true)
 |-- sale_price: double (nullable = true)



### 4.3.2 Data Cleaning

In [33]:
#drop duplikat apabila data tersebut terdapat data yang duplikat
date_dimension = date_dimension.dropDuplicates()

In [34]:
#kita drop semua data kecuali kolom "created_at"
date_dimension = date_dimension.drop("id","order_id","user_id","product_id","inventory_item_id","status","shipped_at","sale_price","returned_at","delivered_at")

In [35]:
#Menampilkan hasil drop data
date_dimension.show()

+-------------------+
|         created_at|
+-------------------+
|2024-10-06 00:52:44|
|2024-05-24 02:59:55|
|2020-07-20 09:12:12|
|2022-08-04 05:57:06|
|2023-07-07 13:53:06|
|2023-06-04 10:36:08|
|2024-08-27 09:45:24|
|2024-04-28 13:35:34|
|2022-02-18 08:37:01|
|2024-12-22 12:26:46|
|2023-02-11 10:50:49|
|2022-07-02 07:50:50|
|2023-10-11 01:37:46|
|2022-09-11 13:33:56|
|2023-06-17 16:59:08|
|2024-12-05 05:18:05|
|2024-04-27 16:36:47|
|2022-10-16 11:44:12|
|2025-01-02 22:40:01|
|2025-01-31 00:33:39|
+-------------------+
only showing top 20 rows



Setelah di drop, data yang tersisa adalah data "created_at" menjadi landasan untuk membuat data "Month", "Quarter", "Half Year" dan "Year"

In [36]:
# untuk membuat data "month" yang berisikan Bulan ke berapa pada data "created_at"
date_dimension = date_dimension.withColumn("month", month(col("created_at")))

In [37]:
# untuk membuat data "quarter" yang berisikan Quarter ke berapa pada data "created_at"
date_dimension = date_dimension.withColumn("quarter", quarter(col("created_at")))

In [38]:
# untuk membuat data "Half" yang berisikan Half Year ke berapa pada data "created_at"
date_dimension = date_dimension.withColumn(
    "half_year",
    when(month(col("created_at")) <= 6, "H1").otherwise("H2")
)

In [39]:
# untuk membuat data "year" yang berisikan informasi tentang tahun pada data "created_at"
date_dimension = date_dimension.withColumn("year", year(col("created_at")))

In [40]:
#membuat unique ID pada "date_dimension"
date_dimension = date_dimension.withColumn("date_id", monotonically_increasing_id()).select("date_id", *date_dimension.columns)

In [41]:
#menampilkan data hasil cleaning
date_dimension.show()

+-------+-------------------+-----+-------+---------+----+
|date_id|         created_at|month|quarter|half_year|year|
+-------+-------------------+-----+-------+---------+----+
|      0|2024-10-06 00:52:44|   10|      4|       H2|2024|
|      1|2024-05-24 02:59:55|    5|      2|       H1|2024|
|      2|2020-07-20 09:12:12|    7|      3|       H2|2020|
|      3|2022-08-04 05:57:06|    8|      3|       H2|2022|
|      4|2023-07-07 13:53:06|    7|      3|       H2|2023|
|      5|2023-06-04 10:36:08|    6|      2|       H1|2023|
|      6|2024-08-27 09:45:24|    8|      3|       H2|2024|
|      7|2024-04-28 13:35:34|    4|      2|       H1|2024|
|      8|2022-02-18 08:37:01|    2|      1|       H1|2022|
|      9|2024-12-22 12:26:46|   12|      4|       H2|2024|
|     10|2023-02-11 10:50:49|    2|      1|       H1|2023|
|     11|2022-07-02 07:50:50|    7|      3|       H2|2022|
|     12|2023-10-11 01:37:46|   10|      4|       H2|2023|
|     13|2022-09-11 13:33:56|    9|      3|       H2|202

Dari data tersebut menunjukkan data berhasil dibuat sesuai dengan keinginan.

In [42]:
# untuk mengecek adanya missing values atau tidak pada date_dimension
date_dimension_mv = date_dimension.select([sum(col(c).isNull().cast("int")).alias(c) for c in date_dimension.columns])

In [43]:
#menampilkan pengecekkan
date_dimension_mv.show()

+-------+----------+-----+-------+---------+----+
|date_id|created_at|month|quarter|half_year|year|
+-------+----------+-----+-------+---------+----+
|      0|         0|    0|      0|        0|   0|
+-------+----------+-----+-------+---------+----+



Setelah dilakukan pengecekkan, tidak adanya missing values pada data "date_dimension".

## 4.4. sales_fact

### 4.4.1. Data Loading

In [44]:
# mengimport data dari "order_items.csv" ke daam dataframe dan diberi nama "sales_fact"
sales_fact = spark.read.csv('thelook_ecommerce.order_items.csv', header=True, inferSchema=True)

In [45]:
#menampilkan isi dari "sales_fact"
sales_fact.show()

+------+--------+-------+----------+-----------------+----------+-------------------+-------------------+-------------------+-----------+------------------+
|    id|order_id|user_id|product_id|inventory_item_id|    status|         created_at|         shipped_at|       delivered_at|returned_at|        sale_price|
+------+--------+-------+----------+-----------------+----------+-------------------+-------------------+-------------------+-----------+------------------+
|127348|   87852|  70237|     14235|           343297| Cancelled|2024-08-02 15:46:40|               NULL|               NULL|       NULL|0.0199999995529651|
| 68311|   47109|  37633|     14235|           184060|  Complete|2022-11-23 14:12:14|2022-11-25 11:43:00|2022-11-28 12:11:00|       NULL|0.0199999995529651|
| 61918|   42739|  34142|     14235|           166877|   Shipped|2022-06-03 02:10:00|2022-06-03 15:19:00|               NULL|       NULL|0.0199999995529651|
|161542|  111381|  89163|     14235|           435667|   S

In [46]:
#menampilkan infromasi dari dati "sales_fact"
sales_fact.printSchema()

root
 |-- id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- inventory_item_id: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- shipped_at: timestamp (nullable = true)
 |-- delivered_at: timestamp (nullable = true)
 |-- returned_at: timestamp (nullable = true)
 |-- sale_price: double (nullable = true)



### 4.4.2. Data Cleaning

In [47]:
#melakukan drop duplikat apabila terdapat data yang duplikat
sales_fact = sales_fact.dropDuplicates()

In [48]:
#untuk menyempurnakan isi dari "sales_fact" maka akan dibuat data 'quantity'. Maka kita buat qty dengan cara seperti berikut
sales_fact_qty = sales_fact.groupBy("order_id", "product_id").agg(count("*").alias("quantity"))

In [49]:
#menunjukkan hasil penambahan qty
sales_fact_qty.show()

+--------+----------+--------+
|order_id|product_id|quantity|
+--------+----------+--------+
|   59731|     13920|       1|
|   96800|     20061|       1|
|   15065|     21617|       1|
|   26791|     24562|       1|
|   36518|      2216|       1|
|    2035|     21910|       1|
|   20422|     20184|       1|
|    3873|     17266|       1|
|  122187|      4481|       1|
|  122482|     16494|       1|
|   83877|     10764|       1|
|   81484|     17560|       1|
|  106446|      3017|       1|
|    4001|     20339|       1|
|  112575|      9344|       1|
|   57559|     24700|       1|
|   74719|      9238|       1|
|    6457|     10951|       1|
|   58521|     18047|       1|
|   41147|      3442|       1|
+--------+----------+--------+
only showing top 20 rows



Data "quantity" berhasil dibuat.

In [50]:
#untuk menyempurnakan isi dari "sales_fact" maka akan dibuat data 'total_amount'. Maka kita hitung total_amount dengan cara seperti berikut
sales_fact_ta = sales_fact.groupBy("order_id", "product_id").agg(sum("sale_price").alias("total_amount"))

In [51]:
#Untuk menampilkan pembuatan data 'total_amount'
sales_fact_ta.show()

+--------+----------+------------------+
|order_id|product_id|      total_amount|
+--------+----------+------------------+
|   59731|     13920|              18.5|
|   96800|     20061|19.989999771118164|
|   15065|     21617|29.989999771118164|
|   26791|     24562|  36.9900016784668|
|   36518|      2216|  41.9900016784668|
|    2035|     21910|  39.9900016784668|
|   20422|     20184| 79.94999694824217|
|    3873|     17266|              72.0|
|  122187|      4481|             189.0|
|  122482|     16494|17.950000762939453|
|   83877|     10764|              10.0|
|   81484|     17560|24.989999771118164|
|  106446|      3017|              25.0|
|    4001|     20339|19.989999771118164|
|  112575|      9344|15.989999771118164|
|   57559|     24700|10.989999771118164|
|   74719|      9238|12.989999771118164|
|    6457|     10951|              16.0|
|   58521|     18047|              18.5|
|   41147|      3442|10.359999656677246|
+--------+----------+------------------+
only showing top

In [52]:
# Data "quantity" dan "total_amount" telah dibuat, lalu kita masukkan ke data "sales_fact" dengan cara join kedua data tersebut.
sales_fact = sales_fact.join(sales_fact_qty, on=["order_id", "product_id"])
sales_fact = sales_fact.join(sales_fact_ta, on=["order_id", "product_id"])

In [53]:
#menampilkan hasil sementara
sales_fact.show()

+--------+----------+------+-------+-----------------+--------+-------------------+-------------------+-------------------+-------------------+------------------+--------+------------------+
|order_id|product_id|    id|user_id|inventory_item_id|  status|         created_at|         shipped_at|       delivered_at|        returned_at|        sale_price|quantity|      total_amount|
+--------+----------+------+-------+-----------------+--------+-------------------+-------------------+-------------------+-------------------+------------------+--------+------------------+
|   14753|     12599| 21463|  11813|            57725|Returned|2024-10-06 00:52:44|2024-10-08 15:52:00|2024-10-09 03:45:00|2024-10-09 16:28:00|               8.0|       1|               8.0|
|   15379|     12671| 22361|  12307|            60163|Returned|2024-05-24 02:59:55|2024-05-26 15:29:00|2024-05-28 04:36:00|2024-05-28 06:31:00|  10.1899995803833|       1|  10.1899995803833|
|   49135|       528| 71220|  39310|         

Data 'quantity' dan 'total_amount' berhasil dimasukkan ke dalam "sales_fact".

In [54]:
# kita akan merubah data "id" menjadi "sales_id" agar menjadi pembeda daripada id id lainnya
sales_fact = sales_fact.withColumnRenamed("id", "sales_id")

In [55]:
#Karena Unique ID dari setiap Dimension ID sudah ada, terkecuali "date_dimension" maka kita akan join "date_dimension" ke "sales_fact"
sales_fact_join = sales_fact.join(date_dimension, on=["created_at"], how="left")

In [56]:
#menampilkan hasil sementara
sales_fact_join.show()

+-------------------+--------+----------+--------+-------+-----------------+--------+-------------------+-------------------+-------------------+------------------+--------+------------------+-----------+-----+-------+---------+----+
|         created_at|order_id|product_id|sales_id|user_id|inventory_item_id|  status|         shipped_at|       delivered_at|        returned_at|        sale_price|quantity|      total_amount|    date_id|month|quarter|half_year|year|
+-------------------+--------+----------+--------+-------+-----------------+--------+-------------------+-------------------+-------------------+------------------+--------+------------------+-----------+-----+-------+---------+----+
|2019-10-19 06:51:38|   53765|       137|   77935|  42957|           210141|Returned|2019-10-20 18:15:00|2019-10-25 16:17:00|2019-10-27 11:10:00|12.989999771118164|       1|12.989999771118164| 8589934602|   10|      4|       H2|2019|
|2019-10-30 01:08:17|   77695|     25173|  112610|  62103|      

Data 'id' sudah berhasil diubah menjadi 'sales_id', dan join data dari "date_dimension" ke "sales_fact" berhasil dilakukan.

Tetapi data terlihat sangat berantakan, maka mari kita rapihkan.

In [57]:
#Kita akan menampilkan data-data yang sekiranya berhubungan hanya pada penjualan saja.
sales_facts = sales_fact_join.select("sales_id", "user_id", "order_id", "product_id", "date_id", "status", "created_at", "delivered_at", "sale_price", "quantity", "total_amount")

In [58]:
#Menampilkan hasil akhir "sales_fact"
sales_facts.show()

+--------+-------+--------+----------+-------+--------+-------------------+-------------------+------------------+--------+------------------+
|sales_id|user_id|order_id|product_id|date_id|  status|         created_at|       delivered_at|        sale_price|quantity|      total_amount|
+--------+-------+--------+----------+-------+--------+-------------------+-------------------+------------------+--------+------------------+
|   21463|  11813|   14753|     12599|      0|Returned|2024-10-06 00:52:44|2024-10-09 03:45:00|               8.0|       1|               8.0|
|   22361|  12307|   15379|     12671|      1|Returned|2024-05-24 02:59:55|2024-05-28 04:36:00|  10.1899995803833|       1|  10.1899995803833|
|   71220|  39310|   49135|       528|      2|Returned|2020-07-20 09:12:12|2020-07-22 06:32:00|10.369999885559082|       1|10.369999885559082|
|   14846|   8161|   10220|      9397|      3|Returned|2022-08-04 05:57:06|2022-08-09 07:51:00|10.989999771118164|       1|10.989999771118164|

Data "sales_fact" berhasil dibuat sesuai keinginan.

# 5. Export ke SQL

In [59]:
# PostgreSQL JDBC Connection
postgres_url = "jdbc:postgresql://host.docker.internal:5432/GC6" #menyambungkan ke database "GC6"
postgres_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [ ]:
# Write DataFrame to PostgreSQL
users_dimension.write.jdbc(url=postgres_url, table="users_dimension", mode="append", properties=postgres_properties)
products_dimension.write.jdbc(url=postgres_url, table="products_dimension", mode="append", properties=postgres_properties)
date_dimension.write.jdbc(url=postgres_url, table="date_dimension", mode="append", properties=postgres_properties)
sales_facts.write.jdbc(url=postgres_url, table="sales_fact", mode="append", properties=postgres_properties)